## Install the required libraries

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [84]:
alerts = pd.read_csv("classified_alerts.csv")

In [85]:
cls = set(alerts['class'])

In [86]:
print(cls)

{'a.c.t.b.r.s.e.customexceptionhandler', 'o.s.w.c.httpservererrorexception.internalservererror', 'o.s.w.c.httpservererrorexception.gatewaytimeout', 'a.c.t.b.r.c.l.loggingaspect', 'a.c.t.b.r.s.m.fromtarget.targetresponsedata', 'a.c.t.b.r.s.e.customexceptionhandler.handlehttpstatuscodeexception', 'j.lang.nullpointerexception', 'o.json.jsonexception', 'o.s.w.c.httpservererrorexception.badgateway', 'c.f.j.d.e.unrecognizedpropertyexception.from', 'a.c.t.b.r.c.u.commonutils'}


In [87]:
class_lst = list(cls)
class_lst.index(class_lst[3])

3

In [88]:
len(alerts)

10912

In [89]:
alerts.columns

Index(['appname', 'alertid', 'class', 'words'], dtype='object')

In [90]:
# Convert class to number
def category2Num(cat):
    global class_lst
    return class_lst.index(cat)
    
alerts['category_num'] = alerts['class'].apply(category2Num)

In [91]:
alerts.head()

,appname,alertid,class,words,category_num
0,spg-service-request-inbound-consumer,a1642502490828,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler requestid ...,0
1,spg-service-request-inbound-consumer,a1642502490829,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler customexce...,0
2,spg-service-request-inbound-consumer,a1642502490838,o.s.w.c.httpservererrorexception.badgateway,oswchttpservererrorexceptionbadgateway 502 bad...,8
3,spg-service-request-inbound-consumer,a1642502490888,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler customexce...,0
4,spg-service-request-inbound-consumer,a1642502490889,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler requestid ...,0


In [92]:
!pip install textblob
!pip install wordcloud

In [93]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt

In [94]:
from string import punctuation
from sklearn import svm
from textblob import Word
from wordcloud import WordCloud
import textblob
from textblob import TextBlob

In [95]:
from imblearn.over_sampling import SMOTE

In [96]:
alerts = alerts.drop(['appname', 'alertid', 'class'], axis=1)
alerts.columns

Index(['words', 'category_num'], dtype='object')

In [97]:
alerts['category_num'].value_counts()

4     4262
5     3595
9     1287
0      544
1      340
10     272
8      204
6      136
2      136
7       68
3       68
Name: category_num, dtype: int64

In [98]:
X = alerts['words']
y = alerts['category_num']

In [99]:
print(X)

0        info actbrsecustomexceptionhandler  requestid ...
1        info actbrsecustomexceptionhandler  customexce...
2        oswchttpservererrorexceptionbadgateway 502 bad...
3        info actbrsecustomexceptionhandler  customexce...
4        info actbrsecustomexceptionhandler  requestid ...
                               ...                        
10907    cfjdexcunrecognizedpropertyexception unrecogni...
10908    warn oswsmmaexceptionhandlerexceptionresolver ...
10909    cfjdexcunrecognizedpropertyexception unrecogni...
10910    cfjdexcunrecognizedpropertyexception unrecogni...
10911    warn oswsmmaexceptionhandlerexceptionresolver ...
Name: words, Length: 10912, dtype: object


In [100]:
alerts.head()

,words,category_num
0,info actbrsecustomexceptionhandler requestid ...,0
1,info actbrsecustomexceptionhandler customexce...,0
2,oswchttpservererrorexceptionbadgateway 502 bad...,8
3,info actbrsecustomexceptionhandler customexce...,0
4,info actbrsecustomexceptionhandler requestid ...,0


In [101]:
my_stop_words = set(['info', 'of', 'and', 'cannot', 'warn', 'is', 'not', 'out', 'doctype', 'html', \
                     'failed', 'failure', 'handle', 'request', 'response', 'throwing', 'line', 'get', \
                     'exception', 'detail', 'details', 'be', 'to', 'in', 'public', 'throws', 'known', \
                     'at', 'from', 'with', 'class', 'marked', 'as', 'it', 'this', 'return', 'returning', \
                     'notfound', 'type', 'title'])

In [102]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= my_stop_words)

In [103]:
#Handle class imbalance
X_features = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=67)
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)

In [111]:
def class_balanced_model_fit(X_train_smote, y_train_smote, X_test_smote, y_test_smote, ml_model, coef_show):
    classification = ml_model.fit(X_train_smote, y_train_smote)
    classification_pred = classification.predict(X_test_smote)
    accuracy = classification.score(X_test_smote, y_test_smote)
    model_performance = classification_report(y_test_smote, classification_pred)
    validation_pred_proba_grad = classification.predict_proba(X_test_smote)
    print(validation_pred_proba_grad)
    #roc_auc = roc_auc_score(y_test_smote, validation_pred_proba_grad[:,1])
    
    print("***** Accuracy of the classification model: ", accuracy, " *******")
    print('')
    print(model_performance)
    #print('')
    #print("***** ROC_AUC score: ", roc_auc, " *******")
    #print("*************************************************")
    
    if coef_show == 1:
        featureNames = tfidf.get_feature_names()
        coef = classification.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : featureNames, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print("************ Top 10 positive features (variables) ************")
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print("************ Top 10 negative features (variables) ************")        
        print(coeff_df.tail(20).to_string(index=False))
    
    return classification

In [112]:
LRClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500),
                                     coef_show=0)

[[2.45507816e-04 1.99552686e-04 2.33615327e-04 ... 2.49809152e-04
  2.33615327e-04 2.04488612e-04]
 [2.45507816e-04 1.99552686e-04 2.33615327e-04 ... 2.49809152e-04
  2.33615327e-04 2.04488612e-04]
 [2.88048208e-04 3.76276193e-04 2.03818370e-04 ... 2.16581119e-04
  2.03818370e-04 3.09435372e-04]
 ...
 [3.35555530e-04 9.76548952e-04 1.87994621e-04 ... 1.98853057e-04
  1.87994621e-04 9.97120952e-01]
 [3.35555530e-04 9.76548952e-04 1.87994621e-04 ... 1.98853057e-04
  1.87994621e-04 9.97120952e-01]
 [3.35555530e-04 9.76548952e-04 1.87994621e-04 ... 1.98853057e-04
  1.87994621e-04 9.97120952e-01]]
***** Accuracy of the classification model:  1.0  *******

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       869
           1       1.00      1.00      1.00       869
           2       1.00      1.00      1.00       869
           3       1.00      1.00      1.00       869
           4       1.00      1.00      1.00       869
           5    

In [79]:
gX_test_smote

<9559x62 sparse matrix of type '<class 'numpy.float64'>'
	with 61115 stored elements in Compressed Sparse Row format>

In [82]:
#Test some random text for classification - Do reviews classification as good or bad sentiment. 
Xfeatu = tfidf.transform(
['cfjdexcunrecognizedpropertyexception unrecognized field error class actbrsmfromtargettargetresponsedata not marked as ignorable 4 known properties status code message incidentid', 
'warn oswsmmaexceptionhandlerexceptionresolver  failure in exceptionhandler public oshttpresponseentity jlangobject actbrsecustomexceptionhandlerhandlehttpstatuscodeexception oswclienthttpstatuscodeexception throws jioioexception',
'info actbrsecustomexceptionhandler  customexceptionhandler handlecustomexception is returning 404 notfound not found errorresource type title badrequest status 400 message cannot deserialize instance of jlangstring out of startarray token'])


In [83]:
print(LRClassModel.predict(Xfeatu))

[4 5 0]


### END OF TELSTRA CODE